# Integer Programming: Fantasy American Football




In this homework, you will use integer programming to select an entry for a fantasy handegg (aka American football) competition. In fantasy sports competitions, each participant selects players to compose a fantasy team, and the goal is to choose a set that maximizes the number of points these players will score in real-life games. Each fantasy entry consists of 10 players. The selection of players is subject to two main types of constraints. The first type restricts the choice of players by position. Namely, a feasible entry contains the following number of players per position (you will find this information for each player in the column 'Position'):

*   1 QB player (quarterback)
*   2 FB player (fullback)
*   2 RB player (running back)
*   3 WR player (wide receiver)
*   2 TE player (tight end)


Each player has a fantasy salary (you will find this information for each player in the column 'Salary'), and there is a budget restriction of \$50,000 over the entry. In other words, the sum of the salaries of the players selected in your entry cannot be larger than \$50,000. The goal is to identify a feasible entry that maximizes the expected number of points (you will find this information for each player in the column 'Points').

Let's suppose that we are solving this problem for a Kaiser Chiefs fan, so  **you must include at least one Kaiser Chiefs (KC) player in your team!**


The extraction and basic filtering of the data have been provided to you (so please don't change that!!!). You will need to solve this exercise in two steps:
- Step 1: Find an optimal solution
- Step 2: Find a second optimal solution by **forbidding the selection of any player that has been selected in the first step**.




**Curiosity**:  Prof. David Bergman, from the OPIM Department, made a lot of money playing this game:
https://today.uconn.edu/2021/01/business-professor-wins-2-5-million-fantasy-football-jackpot-using-concepts-teaches-students/


**Data source (2020: 1-22)**: https://dailyroto.com/nfl-historical-production-fantasy-points-draftkings-fanduel/





## Setup Your Environment/Imports

In [ ]:
# before you do anything...
# mount your drive!
# click folder on the left...
# import modules

%matplotlib inline
from pylab import *

import shutil
import sys
import os.path
import pandas as pd
import numpy as np
from pylab import * # simpler interface to matplotlib

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc
        except:
            pass

assert(shutil.which("cbc") or os.path.isfile("cbc"))

from pyomo.environ import *

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.1 MB/s eta 0:00:00
Selecting previously unselected package coinor-libcoinutils3v5:amd64.
(Reading database ... 123630 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.11.4+repack1-2_amd64.deb ...
Unpacking coinor-libcoinutils3v5:amd64 (2.11.4+repack1-2) ...
Selecting previously unselected package coinor-libosi1v5:amd64.
Preparing to unpack .../1-coinor-libosi1v5_0.108.6+repack1-2_amd64.deb ...
Unpacking coinor-libosi1v5:amd64 (0.108.6+repack1-2) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.17.5+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.17.5+repack1-1) ...
Selecting previously unselected package coinor-libcgl1:amd64.
Preparing to unpack .../3-coinor-libcgl1_0.60.3+repack1-3_amd64.deb ...
Unpacking coinor-libcgl1:amd64 (0.60.3+repack1-3) ..

## Finding the data we need

In [ ]:
# Step 1: copy & paste the shared link you will get from Google Drive
URL = 'https://drive.google.com/file/d/1RBQuzs6yQ74_4ECagt_-Ws_IeTt2oO9X/view?usp=sharing'

# Step 2: copy & paste the command below (this part of the command is fixed)
fixed_path = 'https://drive.google.com/uc?export=download&id='


# Step 3: Extract the file ID from the URL (this part of the command is fixed)
file_path = URL.split('/')[-2]

# Retrieve the CSV data and build a dataframe
df = pd.read_csv(fixed_path + file_path)

# Basic data cleansing (PLEASE DO NOT CHANGE THIS!!!)
del df['DK Points']
del df['DKSal']
df = df[df["Week"] == 12].reset_index(drop=True)
df.rename(columns = {'P': 'Position', 'FDSal': 'Salary', 'FD Points': 'Points'}, inplace = True)

# First lines of the dataframe
df.head()

,Position,Player,Week,Team,Opp,Opp Rank,Opp Position Rank,Salary,Points
0,WR,Tyreek Hill,12,KC,TB,13,8600,7800,60.90
1,RB,Derrick Henry,12,TEN,IND,7,8300,7900,41.50
2,RB,Antonio Gibson,12,WAS,DAL,24,6800,6000,39.60
3,QB,Deshaun Watson,12,HOU,DET,17,8700,7400,36.12
4,WR,Will Fuller V,12,HOU,DET,17,7400,6400,38.10


# Data Extraction

In [ ]:
indices = df.index.values.tolist()

Position = df['Position'].to_list()
Salary = df['Salary'].to_list()
Points = df['Points'].to_list()
Team = df['Team'].to_list()

Teams = df["Team"].unique()
print(len(indices))

572


In [ ]:
print(Teams)

['KC' 'TEN' 'WAS' 'HOU' 'MIN' 'GB' 'NO' 'CLE' 'JAX' 'TB' 'CHI' 'ARI' 'SEA'
 'DAL' 'PHI' 'MIA' 'IND' 'SF' 'LAC' 'DET' 'CAR' 'NYG' 'BAL' 'BUF' 'ATL'
 'LAR' 'NE' 'PIT' 'CIN' 'LV' 'NYJ' 'DEN']


# Formulation

**Define the Objective Function**

$\max \sum\limits_{i = 0}^{n} Points[i]*x_i$ `(objective function)`

**Write the Constraints**

subject to:
* $\sum\limits_{i \in FB } x_i = 2$       `(FB)`
* $\sum\limits_{i \in RB } x_i = 2$       `(RB)`
* $\sum\limits_{i \in WR } x_i = 3$       `(WR)`
* $\sum\limits_{i \in TE } x_i = 2$       `(TE)`
* $\sum\limits_{i = 0}^{n} Salary[i]*x_i \leq 50,000$ `(Salary cap)`

*Variables*
* $x_i \in \{0,1\}$


# Solving the problem

In [ ]:
# declare the model
model = ConcreteModel()

# declare decision variables
model.select = Var(indices, domain=Binary)


# Constraints
model.constraints = ConstraintList()

#   1 QB player
expr = 0
for index in indices:
  if Position[index] == "QB":
    expr += model.select[index]
model.QB = Constraint( expr =  expr == 1)

#   2 FB players
expr = 0
for index in indices:
  if Position[index] == "FB":
    expr += model.select[index]
model.FB = Constraint( expr =  expr == 2)

#   2 RB players
expr = 0
for index in indices:
  if Position[index] == "RB":
    expr += model.select[index]
model.RB = Constraint( expr =  expr == 2)

#   3 WR players
expr = 0
for index in indices:
  if Position[index] == "WR":
    expr += model.select[index]
model.WR = Constraint( expr =  expr == 3)

#   2 TE players
expr = 0
for index in indices:
  if Position[index] == "TE":
    expr += model.select[index]
model.TE = Constraint( expr =  expr == 2)


# At least one KC
expr = 0
for index in indices:
  if Team[index] == "KC":
    expr += model.select[index]
model.KC = Constraint( expr =  expr >= 1)


# Salary cap
expr = 0
for index in indices:
  expr += Salary[index]*model.select[index]
model.Salary = Constraint( expr =  expr <= 50000)


# Objective
expr = 0
for index in indices:
  expr += Points[index]*model.select[index]
model.Points = Objective( expr =  expr, sense = maximize)

# show the model you've created
# model.pprint()

# solve it
SolverFactory('cbc', executable='/usr/bin/cbc').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 288.92
  Upper bound: 288.92
  Number of objectives: 1
  Number of constraints: 7
  Number of variables: 532
  Number of binary variables: 572
  Number of integer variables: 572
  Number of nonzeros: 285
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.04
  Wallclock time: 0.05
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and 

# Print the solution


In [ ]:
# show the results
print("Fantasy Points = ", model.Points())
print("Roster")
ind_list = list()
for index in indices:
  if model.select[index]() is None:
    continue
  if  model.select[index]() > 0:
    ind_list.append(index)

df.iloc[ind_list]

Fantasy Points =  288.91999999999996
Roster


,Position,Player,Week,Team,Opp,Opp Rank,Opp Position Rank,Salary,Points
0,WR,Tyreek Hill,12,KC,TB,13,8600,7800,60.90
1,RB,Derrick Henry,12,TEN,IND,7,8300,7900,41.50
2,RB,Antonio Gibson,12,WAS,DAL,24,6800,6000,39.60
3,QB,Deshaun Watson,12,HOU,DET,17,8700,7400,36.12
4,WR,Will Fuller V,12,HOU,DET,17,7400,6400,38.10
11,WR,Jarvis Landry,12,CLE,JAX,28,5700,5200,31.30
36,TE,Dallas Goedert,12,PHI,SEA,10,6200,4200,20.50
55,TE,Evan Engram,12,NYG,CIN,28,5600,4500,20.90
346,FB,Andy Janovich,12,CLE,JAX,25,4500,0,0.00
410,FB,Danny Vitale,12,NE,ARI,13,4500,0,0.00


In [ ]:
print(ind_list)

[0, 1, 2, 3, 4, 11, 36, 55, 346, 410]


# Expand the model to compute the second entry

In [ ]:
# We are adding a new set of constraints to the model
for index in ind_list:
  model.constraints.add(model.select[index] == 0)

# solve it
SolverFactory('cbc', executable='/usr/bin/cbc').solve(model).write()
# show the results
print("Fantasy Points = ", model.Points())
print("Roster")
ind_list = list()
for index in indices:
  if model.select[index]() is None:
    continue
  if  model.select[index]() > 0:
    ind_list.append(index)

df.iloc[ind_list]

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 218.88
  Upper bound: 218.88
  Number of objectives: 1
  Number of constraints: 7
  Number of variables: 524
  Number of binary variables: 572
  Number of integer variables: 572
  Number of nonzeros: 277
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.07
  Wallclock time: 0.09
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and 

,Position,Player,Week,Team,Opp,Opp Rank,Opp Position Rank,Salary,Points
5,QB,Patrick Mahomes,12,KC,TB,8,9000,8000,35.28
8,RB,Latavius Murray,12,NO,DEN,15,5000,5900,28.60
16,RB,David Montgomery,12,CHI,GB,30,6000,5300,28.30
17,WR,DK Metcalf,12,SEA,PHI,13,8400,7700,30.70
20,WR,Amari Cooper,12,DAL,WAS,2,6900,5700,26.20
25,WR,Deebo Samuel,12,SF,LAR,1,6000,5400,27.30
46,TE,Robert Tonyan,12,GB,CHI,23,5800,3400,17.70
59,TE,Rob Gronkowski,12,TB,KC,13,6200,4400,19.60
170,FB,Kyle Juszczyk,12,SF,LAR,4,4500,4000,5.20
293,FB,Jamize Olawale,12,DAL,WAS,11,4500,0,0.00
